In [1]:
from transformers import pipeline



/Users/neilmacphee/Code/Github/NeilMacPhee/nlp-experiments/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598051905632019},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

Creating the tokenizer

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [4]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


Building the model

In [5]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [6]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


In [7]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [8]:
print(outputs.logits.shape)
#since we have 2 input sentences and two labels

torch.Size([2, 2])


post processing

In [9]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4419e-04]], grad_fn=<SoftmaxBackward0>)


In [10]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

Creating a transformer


In [12]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model
model = BertModel(config)

In [13]:
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.37.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [18]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

In [19]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [20]:
outputs = model(**inputs)

In [25]:
print(outputs)


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3412, -0.1088,  0.1147,  ..., -0.2650,  0.1881, -0.1104],
         [-0.0316, -0.9939,  0.3383,  ..., -0.2290,  0.5234,  0.2093],
         [ 0.1651, -0.8795,  0.5937,  ..., -0.0165, -0.0986,  0.1480],
         ...,
         [-0.1512, -0.5010,  0.1329,  ..., -0.3520, -0.1183,  0.2425],
         [ 0.1559, -0.4795,  0.1415,  ..., -0.4274, -0.2023,  0.2730],
         [ 0.9356, -0.5068,  0.2157,  ..., -0.9903,  0.0920, -0.6285]],

        [[-0.0725,  0.0540, -0.0037,  ...,  0.1450,  0.2381, -0.0164],
         [-0.1619, -0.3062, -0.2282,  ...,  0.3782, -0.1170,  0.1295],
         [-0.1173, -0.1003,  0.1703,  ...,  0.3227, -0.1996,  0.1646],
         ...,
         [-0.1968, -0.3095, -0.2132,  ...,  0.3795, -0.0715,  0.0696],
         [-0.1486, -0.2172, -0.1510,  ...,  0.3222,  0.1034, -0.0320],
         [-0.1927, -0.1105,  0.1996,  ...,  0.4188,  0.2443, -0.0229]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou